In [3]:
import tensorflow.compat.v1 as tf
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K
import tensorflow_hub as hub
import pandas as pd
import numpy as np

In [4]:
tf.disable_eager_execution()

In [5]:
# Let's create input ####
full_training_data = pd.read_csv('full_training_data_noneutral.csv')


X_training_data_imb = full_training_data.tweet_text
Y_training_data_imb = full_training_data.label
full_training_data.head()

,Unnamed: 0,sentiment,tweet_text,label
0,0,negative,Theo Walcott is still shit\u002c watch Rafa an...,0
1,1,negative,its not that I\u2019m a GSP fan\u002c i just h...,0
2,2,negative,Iranian general says Israel\u2019s Iron Dome c...,0
3,5,positive,with J Davlar 11th. Main rivals are team Polan...,1
4,6,negative,Talking about ACT\u2019s && SAT\u2019s\u002c d...,0


In [6]:
list_data = full_training_data["tweet_text"].tolist()

In [7]:
y = full_training_data["label"].tolist()

In [8]:
max_len = 0


for lists in list_data:
    length = len(lists)
    if length > max_len:
        max_len = length

print(max_len)

188


In [9]:
### Now let's create input dataloaders ###

from sklearn.model_selection import train_test_split

texts = list_data
texts = [' '.join(t.split()[:max_len]) for t in texts]
texts = np.array(texts, dtype=object)[:, np.newaxis]

x_train, x_val, y_train, y_val = train_test_split(texts, y, random_state=1992, test_size=0.2)

In [10]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable = True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        #'https://tfhub.dev/google/elmo/2' produce the same error. 
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable,
                               name="{}_module".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(
            K.squeeze(K.cast(x, tf.string), axis=1),
            as_dict=True,
            signature='default',
            )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    Dropout
)

num_classes = 1
batch_size = 512
epochs = 200
learnRate = 0.001

input_text = Input(shape=(1,), dtype="string", name='input_0')
x = ElmoEmbeddingLayer(trainable=False)(input_text)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dense(num_classes, activation="sigmoid")(x)

model = Model(inputs=[input_text], outputs=x)

model.summary()

# 'binary_crossentropy'
model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [13]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=5,batch_size=32)

Train on 7484 samples, validate on 1871 samples
Epoch 1/5


FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Could not find variable elmo_embedding_layer_module/bilm/CNN/b_cnn_3. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status=Not found: Resource localhost/elmo_embedding_layer_module/bilm/CNN/b_cnn_3/class tensorflow::Var does not exist.
	 [[{{node elmo_embedding_layer/elmo_embedding_layer_module_apply_default/bilm/CNN_1/add_3/ReadVariableOp}}]]
	 [[metrics/accuracy/broadcast_weights/assert_broadcastable/AssertGuard/pivot_f/_30/_152]]
  (1) Failed precondition: Could not find variable elmo_embedding_layer_module/bilm/CNN/b_cnn_3. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status=Not found: Resource localhost/elmo_embedding_layer_module/bilm/CNN/b_cnn_3/class tensorflow::Var does not exist.
	 [[{{node elmo_embedding_layer/elm